In [1]:
library(glmnet)
library(SLOPE)
library(bigstep)
library(ggplot2)
library(tidyr)
library(gridExtra)
library(knockoff)

Loading required package: Matrix

Loaded glmnet 4.1-8


Attaching package: ‘tidyr’


The following objects are masked from ‘package:Matrix’:

    expand, pack, unpack




### Computer project

In [2]:
generate.data = function(n = 500, p = 450, k = 5) list(
	X = matrix(data = rnorm(n * p, 0, sd=sqrt(1/n)), nrow = n),
	β = 10 * (1:p %in% seq(1, k))
)

In [3]:
W = function(coefs) {
	n = length(coefs) / 2
	real <- coefs[1:n]
	fake <- coefs[(n+1):(2*n)]
	return( abs(real) - abs(fake) )
} 

In [61]:
W.model = function(alpha) {
	return(function(X, X_k, y) {
		lasso = cv.glmnet(cbind(X, X_k), y, standardize=F, intercept=F, alpha=alpha)
		B = coef(lasso, s='lambda.min')[-1]
		return(W(B))
	})
}

knockoff.ridge = function(X, Xk, y) knockoff.filter(X, y, knockoffs = function(X) Xk, statistic = W.model(0), fdr = 0.2)$selected
knockoff.lasso = function(X, Xk, y) knockoff.filter(X, y, knockoffs = function(X) Xk, statistic = W.model(1), fdr = 0.2)$selected

In [62]:
experiment = function(model, k, n = 500, p = 450, iters=100) {
	data = generate.data(n = n, p = p, k = k)
	X = data$X
	β = data$β
	Xk <- create.second_order(X)

	metrics = function(coefs) return(c(
		power = sum(coefs <= k) / k,
		fdp = sum(coefs > k) / p
	))

	iter = function() {
		y = X %*% β + rnorm(n, 0, sd=2)
		variables = model(X, Xk, y)
		metrics(variables)
		# knocks = knockoff.filter(X, y, knockoffs = function(X) Xk, statistic = model, fdr = 0.2)$selected
		# metrics(knocks)
	}

	rowMeans(replicate(iters, iter()))
}

In [63]:
experiment(knockoff.lasso, 50, iters=100)

power   fdp 
    0     0

In [ ]:
rowMeans(replicate(100, knockoffs.model(alpha = 1, k = 50)))

# uwu


In [ ]:
metrics = function(B) c(
	E1  = sum((B - β)**2),
	E2  = sum((X %*% (B - β))**2),
	FDP = sum(B[21:p] != 0) / sum(B != 0),
	TPP = sum(B[1:20] != 0) / 20
) 

In [ ]:
# use intercept?
ridge = cv.glmnet(X, y, alpha=0, intercept = F)
ridge.metrics = metrics(coef(ridge)[-1,])

In [ ]:
ridge.metrics

#### LASSO

When fitting a cross-validated lasso model using cv.glmnet, two lambda values are commonly reported:

`cv_fit$lambda.min`: The value of lambda that gives the minimum mean cross-validated error. This is often referred to as the "best" lambda because it directly minimizes the prediction error on the validation set.

`cv_fit$lambda.1se`: The largest value of lambda for which the mean cross-validated error is within one standard error of the minimum. This lambda value usually results in a sparser model (fewer non-zero coefficients), potentially improving interpretability and generalization by favoring simpler models.

In [ ]:
lasso = cv.glmnet(X, y, alpha=1)
lasso.min.metrics = metrics(coef(lasso, s = "lambda.min")[-1,])
lasso.1se.metrics = metrics(coef(lasso, s = "lambda.1se")[-1,])

In [ ]:
lasso.min.metrics

In [ ]:
lasso.1se.metrics

In [ ]:
lasso.arg = glmnet(X, y, alpha=1, lambda = qnorm(1 - 0.1/2/p)/n)

In [ ]:
lasso.arg.metrics = metrics(coef(lasso.arg)[-1,])

In [ ]:
lasso.arg.metrics

In [ ]:
lasso.ols = glmnet(X, y, alpha=1, lambda=0)
lasso.ols.metrics = metrics(coef(lasso)[-1,])
lasso.ols.metrics

In [ ]:
glmnet(X, Y, alpha = 1, lambda = 0)

#### SLOPE

In [ ]:
# https://cran.r-project.org/web/packages/SLOPE/vignettes/introduction.html
slope = SLOPE(X, y, lambda = qnorm(1 - seq(1, 950, 1)*0.1/2/p)/n)

In [ ]:
metrics(coef(slope)[,20][-1])

In [ ]:
slope = SLOPE(X, y, lambda=rep(0, 950))
metrics(coef(slope)[,2][-1])